In [2]:
from lxml import etree
import pandas as pd
# import dask.dataframe as dd

In [1]:
file_path = "C:/Users/Настя/YandexDisk-n4skolesnikova/HSE (4 курс)/Выпускная квалификационная работа (ВКР)/trueGibddParser/storage/ДТП_"
regions = [
    1,
    3,
    4,
    5,
    7,
    8,
    10,
    11,
    12,
    14,
    15,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    49,
    50,
    52,
    53,
    54,
    55,
    56,
    57,
    58,
    60,
    61,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80,
    81,
    82,
    83,
    84,
    85,
    86,
    87,
    88,
    89,
    90,
    91,
    92,
    93,
    94,
    95,
    96,
    97,
    98,
    99,
    71100,
    71140,
    10011,
]
years = range(2015, 2025)
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

russians = ['VIS', 'Bronto', 'Lada', 'GAZ', 'ГАЗ', 'Газ', 'Урал', 'ПАЗ', 'КАВЗ', 'ЛиАЗ', 'ГОЛАЗ', 'КАМАЗ', 'НефАЗ', 'VDL', 'Бравис',\
            'УАЗ', 'Ssang', 'ЗИЛ', 'Баз', 'Грейт', 'Opel', 'Volgabus', 'BAW', 'Tar', 'Lifan', 'Geely', 'Блеск', 'Chery', 'Jianghuai', 'Хавтай']

In [78]:
data = []
dropped_rta = 0

for region in regions:
    for year in years:
        for month in months:
            try:
                file_name = file_path + str(region) + '_' + str(year) + '_' + month + '.xml'
                tree = etree.parse(file_name)

                root = tree.getroot()

                for tab in root.iter('tab'):
                    kuch = tab.find('KUCH')
                    if kuch is not None and kuch.text and kuch.text.isdigit():
                        if int(kuch.text) >= 10:
                            dropped_rta += 1
                            continue
                    kts = tab.find('KTS')
                    if kts is not None and kts.text and kts.text.isdigit():
                        if int(kts.text) >= 5:
                            dropped_rta += 1
                            continue

                    record = {}
                    for element in tab.iter():
                        if element.tag not in {'tab', 'infoDtp', 'ts_info', 'ts_uch', 'uchInfo'}:
                            record[element.tag] = element.text
                        elif element.tag == 'infoDtp':
                            for info in element.iter():
                                if info.tag not in {'infoDtp', 'ts_info', 'ts_uch', 'uchInfo'}:
                                    record[info.tag] = info.text


                    # binary features
                    record['any TV technical failures'] = 0
                    record['any non-private vehicles'] = 0
                    record['any russian vehicles'] = 0

                    record['any vehicles older than 1990'] = 0
                    record['any vehicles older than 2000'] = 0
                    record['any vehicles older than 2010'] = 0

                    record['any white vehicles'] = 0
                    record['any black vehicles'] = 0
                    record['any colored vehicles'] = 0

                    record['any front-wheel-drive vehicles'] = 0
                    record['any rear-wheel-drive vehicles'] = 0
                    record['any four-wheel-drive vehicles'] = 0

                    record['any A-class vehicles'] = 0
                    record['any B-class vehicles'] = 0
                    record['any C-class vehicles'] = 0
                    record['any D-class vehicles'] = 0
                    record['any E-class vehicles'] = 0
                    record['any S-class vehicles'] = 0
                    record['any non-car vehicles'] = 0
    

                    record['any drunk passengers'] = 0
                    record['any drunk drivers'] = 0
                    record['any female drivers'] = 0
                    record['any escaped participants'] = 0
                    record['any pedestrians'] = 0
                    record['any cyclists'] = 0

                    record['any concomitant traffic violations'] = 0
                    record['any injuries without safety belt'] = 0
                    record['any child restraints'] = 0


                    # non-binary features
                    record['number of guilty objects'] = 0
                    number_of_guilty_drivers = 0
                    record['fatal traffic violation(s)'] = None
                    record['number of fatal traffic violations'] = 0
                    record['bodily harm'] = None

                    record['guilty drivers\' average experience'] = None
                    guilty_drivers_experience = 0
                    record['drivers\' average experience'] = None
                    drivers_experience = 0


                    violations = set()
                    injuries = set()

                    for ts_info in tab.iter('ts_info'):

                        failures = ts_info.find('t_n')
                        if failures is not None and failures.text and failures.text != 'Технические неисправности отсутствуют':
                            record['any TV technical failures'] = 1

                        property = ts_info.find('f_sob')
                        if property is not None and property.text and property.text != 'Частная собственность':
                            record['any non-private vehicles'] = 1

                        year_of_release = ts_info.find('g_v')
                        if year_of_release is not None and year_of_release.text and year_of_release.text.isdigit():
                            if int(year_of_release.text) < 1990:
                                record['any vehicles older than 1990'] = 1
                            elif int(year_of_release.text) < 2000:
                                record['any vehicles older than 2000'] = 1
                            elif int(year_of_release.text) < 2010:
                                record['any vehicles older than 2010'] = 1

                        color = ts_info.find('color')
                        if color is not None and color.text:
                            if color.text == 'Белый':
                                record['any white vehicles'] = 1
                            elif color.text == 'Черный':
                                record['any black vehicles'] = 1
                            else:
                                record['any colored vehicles'] = 1

                        wheel = ts_info.find('r_rul')
                        if wheel is not None:
                            if wheel.text == 'С передним приводом':
                                record['any front-wheel-drive vehicles'] = 1
                            elif wheel.text == 'С задним приводом':
                                record['any rear-wheel-drive vehicles'] = 1
                            elif wheel.text == 'Полноприводные':
                                record['any four-wheel-drive vehicles'] = 1

                        vehicle_class = ts_info.find('t_ts')
                        if vehicle_class is not None and vehicle_class.text:
                            if vehicle_class.text == 'А-класс (особо малый) до 3,5 м':
                                record['any A-class vehicles'] = 1
                            elif vehicle_class.text == 'В-класс (малый) до 3,9 м':
                                record['any B-class vehicles'] = 1
                            elif vehicle_class.text == 'С-класс (малый средний, компактный) до 4,3 м':
                                record['any C-class vehicles'] = 1
                            elif vehicle_class.text == 'D-класс (средний) до 4,6 м':
                                record['any D-class vehicles'] = 1
                            elif vehicle_class.text == 'Е-класс (высший средний, бизнес-класс) до 4,9 м':
                                record['any E-class vehicles'] = 1
                            elif vehicle_class.text == 'S-класс (высший, представительский класс) более 4,9 м':
                                record['any S-class vehicles'] = 1

                        model = ts_info.find('m_ts')
                        if model is not None and model.text:
                            for rus in russians:
                                if rus in model:
                                    record['any russian vehicles'] = 1
                                    continue


                        is_vehicle_guilty = 0

                        for ts_uch in ts_info.iter('ts_uch'):

                            drunk = ts_uch.find('ALCO')                                
                            participant = ts_uch.find('k_UCH')
                            gender = ts_uch.find('POL')

                            if participant is not None:
                                if participant.text == 'Водитель' and drunk is not None and drunk.text:
                                    record['any drunk drivers'] = 1
                                elif participant.text == 'Пассажир' and drunk is not None and drunk.text:
                                    record['any drunk passengers'] = 1
                                elif participant.text == 'Велосипедист':
                                    record['any cyclists'] = 1
                                
                                if gender is not None and gender.text == 'Женский' and participant.text == 'Водитель':
                                    record['any female drivers'] = 1

                            leaving = ts_uch.find('s_SM')
                            if leaving is not None and leaving.text and leaving.text != 'Нет (не скрывался)':
                                record['any escaped participants'] = 1

                            concomitant_violations = ts_uch.find('SOP_NPDD')
                            if concomitant_violations is not None and concomitant_violations.text and concomitant_violations.text != 'Нет нарушений':
                                record['any concomitant traffic violations'] = 1

                            child = ts_uch.find('s_SEAT_GROUP')
                            if child is not None and child.text:
                                record['any child restraints'] = 1

                            safety_belt = ts_uch.find('SAFETY_BELT')
                            injury = ts_uch.find('s_T')
                            if injury is not None and injury.text and injury.text != 'Не пострадал':
                                injuries.add(injury.text)
                                if safety_belt is not None and safety_belt.text == 'Нет':
                                    record['any injuries without safety belt'] = 1

                            # guilty = ts_uch.find('NPDD')
                            # if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
                            #     is_vehicle_guilty = 1
                            #     violations.add(guilty.text)

                            is_obj_guily = 0
                            for guilty in ts_uch.iter('NPDD'):
                                if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
                                    is_vehicle_guilty = 1
                                    violations.add(guilty.text)

                            experience = ts_uch.find('v_ST')
                            if experience is not None and experience.text and experience.text.isdigit():
                                drivers_experience += int(experience.text)
                                if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
                                    guilty_drivers_experience += int(experience.text)

                        record['number of guilty objects'] += is_vehicle_guilty
                        number_of_guilty_drivers += is_vehicle_guilty

                    if number_of_guilty_drivers != 0:  
                        record['guilty drivers\' average experience'] = guilty_drivers_experience / number_of_guilty_drivers
                    record['drivers\' average experience'] = drivers_experience / int(kts.text)

                    pedestrian = tab.find('uchInfo')
                    if pedestrian is not None:
                        record['any pedestrians'] = 1


                    guilty_objects = 0

                    for uchInfo in tab.iter('uchInfo'):

                        leaving = uchInfo.find('s_SM')
                        if leaving is not None and leaving.text and leaving.text != 'Нет (не скрывался)':
                            record['any escaped participants'] = 1

                        concomitant_violations = uchInfo.find('SOP_NPDD')
                        if concomitant_violations is not None and concomitant_violations.text and concomitant_violations.text != 'Нет нарушений':
                            record['any concomitant traffic violations'] = 1

                        # guilty = uchInfo.find('NPDD')
                        # if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
                        #     guilty_objects += 1
                        #     # record['number of fatal traffic violations'] += 
                        #     violations.add(guilty.text)

                        is_obj_guily = 0
                        for guilty in uchInfo.iter('NPDD'):
                            if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
                                is_obj_guily = 1
                                violations.add(guilty.text)
                        guilty_objects += is_obj_guily

                        injury = uchInfo.find('s_T')
                        if injury is not None and injury.text and injury.text != 'Не пострадал':
                            injuries.add(injury.text)

                    record['number of guilty objects'] += guilty_objects
                    record['fatal traffic violation(s)'] = violations
                    record['number of fatal traffic violations'] = len(violations)

                    if len(injuries) != 0:
                        record['bodily harm'] = injuries
                    else:
                        record['bodily harm'] = {'Не пострадал'}

                    data.append(record)

            except OSError:
                continue

print(dropped_rta)
df_correct = pd.DataFrame(data)
df_correct.to_csv('DTP_DATA_CORRECT.csv')
df_correct

10459


,DTPV,date,district,EMTP_NUMBER,CHOM,COORD_L,COORD_W,dor,dor_k,dor_z,...,any cyclists,any concomitant traffic violations,any injuries without safety belt,any child restraints,number of guilty objects,fatal traffic violation(s),number of fatal traffic violations,bodily harm,guilty drivers' average experience,drivers' average experience
0,Столкновение,31.01.2015,ЖЕЛЕЗНОДОРОЖНЫЙ,None,Движение частично перекрыто,83.740833,53.333056,None,None,Не указано,...,0,0,0,0,1,{Другие нарушения ПДД водителем},1,"{Раненый, находящийся (находившийся) на амбул...",15.0,13.0
1,Наезд на пешехода,31.01.2015,АЛЕЙСКИЙ,None,Режим движения не изменялся,82.462778,52.245833,А-322 Барнаул - Рубцовск - граница с Республик...,3,Федеральная (дорога федерального значения),...,0,1,0,0,1,{Нахождение на проезжей части без цели её пере...,1,"{Раненый, находящийся (находившийся) на стацио...",NaN,5.0
2,Наезд на пешехода,31.01.2015,ИНДУСТРИАЛЬНЫЙ,None,Движение частично перекрыто,83.501944,53.274167,None,None,Не указано,...,0,0,0,0,2,{Переход через проезжую часть вне пешеходного ...,1,"{Раненый, находящийся (находившийся) на амбул...",NaN,17.0
3,Наезд на пешехода,31.01.2015,ЦЕНТРАЛЬНЫЙ,None,Режим движения не изменялся,83.251667,53.285278,None,None,Не указано,...,0,0,0,0,1,{Переход через проезжую часть вне пешеходного ...,1,"{Раненый, находящийся (находившийся) на амбул...",NaN,27.0
4,Наезд на пешехода,31.01.2015,ИНДУСТРИАЛЬНЫЙ,None,Движение частично перекрыто,83.708889,53.318333,None,None,Не указано,...,0,0,0,0,1,{Переход через проезжую часть в неустановленно...,1,"{Раненый, находящийся (находившийся) на стацио...",NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400910,Наезд на стоящее ТС,09.04.2024,Заполярный район,510000158,Режим движения не изменялся,53.364973,67.648813,None,None,"Местного значения (дорога местного значения, в...",...,0,1,1,0,1,{Несоответствие скорости конкретным условиям д...,1,"{Раненый, находящийся (находившийся) на амбула...",95.0,57.0
1400911,Опрокидывание,07.04.2024,Заполярный район,510000154,Режим движения не изменялся,64.919689,69.244707,None,None,"Местного значения (дорога местного значения, в...",...,0,1,1,0,1,{Несоответствие скорости конкретным условиям д...,1,{Получил телесные повреждения с показанием к л...,95.0,95.0
1400912,Столкновение,07.05.2024,Нарьян-Мар,510000212,Движение частично перекрыто,53.046702,67.648548,None,None,"Местного значения (дорога местного значения, в...",...,0,1,0,0,1,{Неправильный выбор дистанции},1,"{Раненый, находящийся (находившийся) на амбула...",27.0,24.5
1400913,Опрокидывание,16.06.2024,Заполярный район,510000245,Режим движения не изменялся,54.095821,67.629532,"г. Нарьян-Мар - г. Усинск, участок г. Нарьян-М...",6,Региональная или межмуниципальная (дорога реги...,...,0,1,1,0,1,{Несоответствие скорости конкретным условиям д...,1,{Получил телесные повреждения с показанием к л...,8.0,8.0


## Intermediate attempt

In [74]:
data = []
file_name = file_path + str(38) + '_' + str(2022) + '_' + '09' + '.xml'
tree = etree.parse(file_name)

root = tree.getroot()

for tab in root.iter('tab'):
    kuch = tab.find('KUCH')
    if kuch is not None and kuch.text and kuch.text.isdigit():
        if int(kuch.text) >= 10:
            dropped_rta += 1
            continue
    kts = tab.find('KTS')
    if kts is not None and kts.text and kts.text.isdigit():
        if int(kts.text) >= 5:
            dropped_rta += 1
            continue

    record = {}
    for element in tab.iter():
        if element.tag not in {'tab', 'infoDtp', 'ts_info', 'ts_uch', 'uchInfo'}:
            record[element.tag] = element.text
        elif element.tag == 'infoDtp':
            for info in element.iter():
                if info.tag not in {'infoDtp', 'ts_info', 'ts_uch', 'uchInfo'}:
                    record[info.tag] = info.text


    # binary features
    record['any TV technical failures'] = 0
    record['any non-private vehicles'] = 0
    record['any russian vehicles'] = 0

    record['any vehicles older than 1990'] = 0
    record['any vehicles older than 2000'] = 0
    record['any vehicles older than 2010'] = 0

    record['any white vehicles'] = 0
    record['any black vehicles'] = 0
    record['any colored vehicles'] = 0

    record['any front-wheel-drive vehicles'] = 0
    record['any rear-wheel-drive vehicles'] = 0
    record['any four-wheel-drive vehicles'] = 0

    record['any A-class vehicles'] = 0
    record['any B-class vehicles'] = 0
    record['any C-class vehicles'] = 0
    record['any D-class vehicles'] = 0
    record['any E-class vehicles'] = 0
    record['any S-class vehicles'] = 0
    record['any non-car vehicles'] = 0


    record['any drunk passengers'] = 0
    record['any drunk drivers'] = 0
    record['any female drivers'] = 0
    record['any escaped participants'] = 0
    record['any pedestrians'] = 0
    record['any cyclists'] = 0

    record['any concomitant traffic violations'] = 0
    record['any injuries without safety belt'] = 0
    record['any child restraints'] = 0


    # non-binary features
    record['number of guilty objects'] = 0
    number_of_guilty_drivers = 0
    record['fatal traffic violation(s)'] = None
    record['number of fatal traffic violations'] = 0
    record['bodily harm'] = None

    record['guilty drivers\' average experience'] = None
    guilty_drivers_experience = 0
    record['drivers\' average experience'] = None
    drivers_experience = 0


    violations = set()
    injuries = set()

    for ts_info in tab.iter('ts_info'):

        failures = ts_info.find('t_n')
        if failures is not None and failures.text and failures.text != 'Технические неисправности отсутствуют':
            record['any TV technical failures'] = 1

        property = ts_info.find('f_sob')
        if property is not None and property.text and property.text != 'Частная собственность':
            record['any non-private vehicles'] = 1

        year_of_release = ts_info.find('g_v')
        if year_of_release is not None and year_of_release.text and year_of_release.text.isdigit():
            if int(year_of_release.text) < 1990:
                record['any vehicles older than 1990'] = 1
            elif int(year_of_release.text) < 2000:
                record['any vehicles older than 2000'] = 1
            elif int(year_of_release.text) < 2010:
                record['any vehicles older than 2010'] = 1

        color = ts_info.find('color')
        if color is not None and color.text:
            if color.text == 'Белый':
                record['any white vehicles'] = 1
            elif color.text == 'Черный':
                record['any black vehicles'] = 1
            else:
                record['any colored vehicles'] = 1

        wheel = ts_info.find('r_rul')
        if wheel is not None:
            if wheel.text == 'С передним приводом':
                record['any front-wheel-drive vehicles'] = 1
            elif wheel.text == 'С задним приводом':
                record['any rear-wheel-drive vehicles'] = 1
            elif wheel.text == 'Полноприводные':
                record['any four-wheel-drive vehicles'] = 1

        vehicle_class = ts_info.find('t_ts')
        if vehicle_class is not None and vehicle_class.text:
            if vehicle_class.text == 'А-класс (особо малый) до 3,5 м':
                record['any A-class vehicles'] = 1
            elif vehicle_class.text == 'В-класс (малый) до 3,9 м':
                record['any B-class vehicles'] = 1
            elif vehicle_class.text == 'С-класс (малый средний, компактный) до 4,3 м':
                record['any C-class vehicles'] = 1
            elif vehicle_class.text == 'D-класс (средний) до 4,6 м':
                record['any D-class vehicles'] = 1
            elif vehicle_class.text == 'Е-класс (высший средний, бизнес-класс) до 4,9 м':
                record['any E-class vehicles'] = 1
            elif vehicle_class.text == 'S-класс (высший, представительский класс) более 4,9 м':
                record['any S-class vehicles'] = 1

        model = ts_info.find('m_ts')
        if model is not None and model.text:
            for rus in russians:
                if rus in model:
                    record['any russian vehicles'] = 1
                    continue


        is_vehicle_guilty = 0

        for ts_uch in ts_info.iter('ts_uch'):

            drunk = ts_uch.find('ALCO')                                
            participant = ts_uch.find('k_UCH')
            gender = ts_uch.find('POL')

            if participant is not None:
                if participant.text == 'Водитель' and drunk is not None and drunk.text:
                    record['any drunk drivers'] = 1
                elif participant.text == 'Пассажир' and drunk is not None and drunk.text:
                    record['any drunk passengers'] = 1
                elif participant.text == 'Велосипедист':
                    record['any cyclists'] = 1
                
                if gender is not None and gender.text == 'Женский' and participant.text == 'Водитель':
                    record['any female drivers'] = 1

            leaving = ts_uch.find('s_SM')
            if leaving is not None and leaving.text and leaving.text != 'Нет (не скрывался)':
                record['any escaped participants'] = 1

            concomitant_violations = ts_uch.find('SOP_NPDD')
            if concomitant_violations is not None and concomitant_violations.text and concomitant_violations.text != 'Нет нарушений':
                record['any concomitant traffic violations'] = 1

            child = ts_uch.find('s_SEAT_GROUP')
            if child is not None and child.text:
                record['any child restraints'] = 1

            safety_belt = ts_uch.find('SAFETY_BELT')
            injury = ts_uch.find('s_T')
            if injury is not None and injury.text and injury.text != 'Не пострадал':
                injuries.add(injury.text)
                if safety_belt is not None and safety_belt.text == 'Нет':
                    record['any injuries without safety belt'] = 1

            # guilty = ts_uch.find('NPDD')
            # if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
            #     is_vehicle_guilty = 1
            #     violations.add(guilty.text)

            is_obj_guily = 0
            for guilty in ts_uch.iter('NPDD'):
                if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
                    is_vehicle_guilty = 1
                    violations.add(guilty.text)

            experience = ts_uch.find('v_ST')
            if experience is not None and experience.text and experience.text.isdigit():
                drivers_experience += int(experience.text)
                if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
                    guilty_drivers_experience += int(experience.text)

        record['number of guilty objects'] += is_vehicle_guilty
        number_of_guilty_drivers += is_vehicle_guilty

    if number_of_guilty_drivers != 0:  
        record['guilty drivers\' average experience'] = guilty_drivers_experience / number_of_guilty_drivers
    record['drivers\' average experience'] = drivers_experience / int(kts.text)

    pedestrian = tab.find('uchInfo')
    if pedestrian is not None:
        record['any pedestrians'] = 1


    guilty_objects = 0

    for uchInfo in tab.iter('uchInfo'):

        leaving = uchInfo.find('s_SM')
        if leaving is not None and leaving.text and leaving.text != 'Нет (не скрывался)':
            record['any escaped participants'] = 1

        concomitant_violations = uchInfo.find('SOP_NPDD')
        if concomitant_violations is not None and concomitant_violations.text and concomitant_violations.text != 'Нет нарушений':
            record['any concomitant traffic violations'] = 1

        # guilty = uchInfo.find('NPDD')
        # if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
        #     guilty_objects += 1
        #     # record['number of fatal traffic violations'] += 
        #     violations.add(guilty.text)

        is_obj_guily = 0
        for guilty in uchInfo.iter('NPDD'):
            if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
                is_obj_guily = 1
                violations.add(guilty.text)
        guilty_objects += is_obj_guily

        injury = uchInfo.find('s_T')
        if injury is not None and injury.text and injury.text != 'Не пострадал':
            injuries.add(injury.text)                     

    record['number of guilty objects'] += guilty_objects
    record['fatal traffic violation(s)'] = violations
    record['number of fatal traffic violations'] = len(violations)

    if len(injuries) != 0:
        record['bodily harm'] = injuries
    else:
        record['bodily harm'] = {'Не пострадал'}

    data.append(record)


df = pd.DataFrame(data)
df.head(3)
# df.to_csv('DTP_DATA_CORRECT.csv')

,DTPV,date,district,EMTP_NUMBER,CHOM,COORD_L,COORD_W,dor,dor_k,dor_z,...,any cyclists,any concomitant traffic violations,any injuries without safety belt,any child restraints,number of guilty objects,fatal traffic violation(s),number of fatal traffic violations,bodily harm,guilty drivers' average experience,drivers' average experience
0,Столкновение,30.09.2022,Курский,380006918,Движение частично перекрыто,36.025114,51.900488,М-2 Крым Москва - Тула - Орел - Курск - Белгор...,4,Федеральная (дорога федерального значения),...,0,0,1,0,1,{Выезд на полосу встречного движения},1,{Получил телесные повреждения с показанием к л...,3.0,20.5
1,Столкновение,29.09.2022,Медвенский,380006871,Режим движения не изменялся,36.136211,51.420179,М-2 Крым Москва - Тула - Орел - Курск - Белгор...,4,Федеральная (дорога федерального значения),...,0,1,1,0,1,{Несоблюдение очередности проезда},1,"{Раненый, находящийся (находившийся) на стацио...",49.0,72.0
2,Наезд на пешехода,29.09.2022,Сеймский,380006876,Режим движения не изменялся,36.160115,51.673214,None,None,"Местного значения (дорога местного значения, в...",...,0,1,0,0,1,{Непредоставление преимущества в движении пеше...,1,"{Раненый, находящийся (находившийся) на амбула...",8.0,8.0


In [75]:
df.columns

Index(['DTPV', 'date', 'district', 'EMTP_NUMBER', 'CHOM', 'COORD_L', 'COORD_W',
       'dor', 'dor_k', 'dor_z', 'factor', 'house', 'k_ul', 'km', 'm', 'NP',
       'ndu', 'OBJ_DTP', 'osv', 's_dtp', 's_pch', 'sdor', 'spog', 'street',
       'color', 'f_sob', 'g_v', 'm_pov', 'm_ts', 'marka_ts', 'n_ts', 'o_pf',
       'r_rul', 't_n', 't_ts', 'ts_s', 'ALCO', 'INJURED_CARD_ID', 'k_UCH',
       'NPDD', 'n_UCH', 'POL', 'SAFETY_BELT', 'SOP_NPDD', 's_SEAT_GROUP',
       's_SM', 's_T', 'v_ST', 'KTS', 'KUCH', 'kartId', 'POG', 'RAN', 'rowNum',
       'time', 'any TV technical failures', 'any non-private vehicles',
       'any russian vehicles', 'any vehicles older than 1990',
       'any vehicles older than 2000', 'any vehicles older than 2010',
       'any white vehicles', 'any black vehicles', 'any colored vehicles',
       'any front-wheel-drive vehicles', 'any rear-wheel-drive vehicles',
       'any four-wheel-drive vehicles', 'any A-class vehicles',
       'any B-class vehicles', 'any C-class 

In [71]:
df['bodily harm']

0     {Получил телесные повреждения с показанием к л...
1     {Раненый, находящийся (находившийся) на стацио...
2     {Раненый, находящийся (находившийся) на амбула...
3     {Раненый, находящийся (находившийся) на амбула...
4     {Раненый, находящийся (находившийся) на амбула...
                            ...                        
95    {Скончался на месте ДТП до приезда скорой меди...
96    {Раненый, находящийся (находившийся) на амбула...
97    {Раненый, находящийся (находившийся) на амбула...
98    {Раненый, находящийся (находившийся) на амбула...
99    {Получил телесные повреждения с показанием к л...
Name: bodily harm, Length: 100, dtype: object

In [76]:
st = set()
for i in range(0, 100):
    st.add(len(df['bodily harm'][i]))

In [77]:
st

{1, 2}